In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [28]:
data_reviews = pd.read_csv('data/Books_rating.csv')

In [29]:
data_books = pd.read_csv('data/books_data.csv')

In [30]:
print("First dataset (data_rating) nan values: ", data_reviews.isnull().sum())
print("Second dataset (data) nan values: ", data_books.isnull().sum())

First dataset (data_rating) nan values:  Id                          0
Title                     208
Price                 2518829
User_id                561787
profileName            561905
review/helpfulness          0
review/score                0
review/time                 0
review/summary            407
review/text                 8
dtype: int64
Second dataset (data) nan values:  Title                 1
description       68442
authors           31413
image             52075
previewLink       23836
publisher         75886
publishedDate     25305
infoLink          23836
categories        41199
ratingsCount     162652
dtype: int64


In [31]:
data_reviews = data_reviews.dropna(subset=['Title', 'User_id'])
data_books = data_books.dropna(subset=['Title'])

In [32]:
# Unisco i due dataset
data_books['Title'] = data_books['Title'].str.lower()
data_reviews['Title'] = data_reviews['Title'].str.lower()

merged_data = pd.merge(data_reviews, data_books, on='Title', how='left')
merged_data['Unified_Id'] = merged_data.groupby('Title').ngroup() + 1

In [33]:
print(merged_data.head())

           Id                           Title  Price         User_id  \
0  1882931173  its only art if its well hung!    NaN   AVCGYZL8FQQTD   
1  0826414346        dr. seuss: american icon    NaN  A30TK6U7DNS82R   
2  0826414346        dr. seuss: american icon    NaN  A3UH4UZ4RSVO82   
3  0826414346        dr. seuss: american icon    NaN  A2MVUWT453QH61   
4  0826414346        dr. seuss: american icon    NaN  A22X4XUPKF66MR   

                          profileName review/helpfulness  review/score  \
0               Jim of Oz "jim-of-oz"                7/7           4.0   
1                       Kevin Killian              10/10           5.0   
2                        John Granger              10/11           5.0   
3  Roy E. Perry "amateur philosopher"                7/7           4.0   
4     D. H. Richards "ninthwavestore"                3/3           4.0   

   review/time                                   review/summary  \
0    940636800           Nice collection of Julie Strai

In [34]:
# controllo se ci sono Nan in unified_id
print("Unified_Id nan values: ", merged_data['Unified_Id'].isnull().sum())

Unified_Id nan values:  0


In [35]:
print("merged_data shape: ", merged_data.shape)

merged_data shape:  (2687063, 20)


In [39]:
min_reviews_book = 20
min_reviews_user = 20

while True:
    # Filtra per libri con almeno 20 recensioni
    book_review_counts = filtered_data['Unified_Id'].value_counts()
    selected_books = book_review_counts[book_review_counts >= min_reviews_book].index
    filtered_data = filtered_data[filtered_data['Unified_Id'].isin(selected_books)]
    
    # Filtra per utenti con almeno 10 recensioni
    user_review_counts = filtered_data['User_id'].value_counts()
    selected_users = user_review_counts[user_review_counts >= min_reviews_user].index
    filtered_data = filtered_data[filtered_data['User_id'].isin(selected_users)]
    
    # Controlla se i criteri sono soddisfatti
    if (
        filtered_data['Unified_Id'].value_counts().min() >= min_reviews_book and
        filtered_data['User_id'].value_counts().min() >= min_reviews_user
    ):
        break

# Controllo finale dei criteri
print("Data shape after filtering:", filtered_data.shape)
print("Number of unique books (Unified_Id) after filtering:", filtered_data['Unified_Id'].nunique())
print("Number of unique users (User_id) after filtering:", filtered_data['User_id'].nunique())
print("Min reviews per book: ", filtered_data['Unified_Id'].value_counts().min())
print("Min reviews per user: ", filtered_data['User_id'].value_counts().min())

Data shape after filtering: (428913, 20)
Number of unique books (Unified_Id) after filtering: 3602
Number of unique users (User_id) after filtering: 9962
Min reviews per book:  20
Min reviews per user:  20


In [48]:
print(filtered_data.head())

             Id                                              Title  Price  \
649  B000MCKQRS  cruel and unusual (g k hall large print book s...    NaN   
651  B000MCKQRS  cruel and unusual (g k hall large print book s...    NaN   
653  B000MCKQRS  cruel and unusual (g k hall large print book s...    NaN   
660  B000MCKQRS  cruel and unusual (g k hall large print book s...    NaN   
662  B000MCKQRS  cruel and unusual (g k hall large print book s...    NaN   

            User_id                            profileName review/helpfulness  \
649  A2HFL26OQJ2T9X                                 H. Jin                0/0   
651  A2YLBDY64IGTBJ        Ruth Thompson "Weaver of Tales"                0/0   
653  A39650P2CZUUC9  Bonnie Brody "Book Lover and Knitter"                0/0   
660     AFVQZQ8PW0L                       Harriet Klausner                2/3   
662  A2OI5RQC4FNKKE                               J. Evans                2/3   

     review/score  review/time  \
649           3.

In [49]:
filtered_data = filtered_data.groupby(['User_id', 'Unified_Id'], as_index=False).agg({
    'review/score': 'mean',
    'Title': 'first',
    'profileName': 'first',
    'review/helpfulness': 'first',
    'review/summary': 'first',
    'review/text': 'first',
    'description': 'first',
    'authors': 'first',
    'publisher': 'first',
    'categories': 'first',
    'ratingsCount': 'first',
    'publishedDate': 'first'
})

In [50]:
print(filtered_data.shape)

(198485, 14)


In [52]:
print("Min reviews per book: ", filtered_data['Unified_Id'].value_counts().min())
print("Min reviews per user: ", filtered_data['User_id'].value_counts().min())

Min reviews per book:  1
Min reviews per user:  1


In [53]:
# filtra di nuovo per eliminare i libri con meno di 20 recensioni e gli utenti con meno di 20 recensioni
while True:
    # Filtra per libri con almeno min_reviews_book recensioni
    book_review_counts = filtered_data['Unified_Id'].value_counts()
    selected_books = book_review_counts[book_review_counts >= min_reviews_book].index
    filtered_data = filtered_data[filtered_data['Unified_Id'].isin(selected_books)]
    
    # Filtra per utenti con almeno min_reviews_user recensioni
    user_review_counts = filtered_data['User_id'].value_counts()
    selected_users = user_review_counts[user_review_counts >= min_reviews_user].index
    filtered_data = filtered_data[filtered_data['User_id'].isin(selected_users)]
    
    # Controlla se i criteri sono soddisfatti
    if (
        filtered_data['Unified_Id'].value_counts().min() >= min_reviews_book and
        filtered_data['User_id'].value_counts().min() >= min_reviews_user
    ):
        break

# Controllo finale dei criteri
print("Data shape after final filtering:", filtered_data.shape)
print("Number of unique books (Unified_Id) after filtering:", filtered_data['Unified_Id'].nunique())
print("Number of unique users (User_id) after filtering:", filtered_data['User_id'].nunique())
print("Min reviews per book: ", filtered_data['Unified_Id'].value_counts().min())
print("Min reviews per user: ", filtered_data['User_id'].value_counts().min())

Data shape after final filtering: (93953, 14)
Number of unique books (Unified_Id) after filtering: 1774
Number of unique users (User_id) after filtering: 2206
Min reviews per book:  20
Min reviews per user:  20


In [54]:
# stampa 3 id univoci
print(filtered_data['Unified_Id'].unique()[:3])

[7120 7121 9332]


In [55]:
# estrai i campioni relativi ai primi 3 id univoci
excluded_books = [int(id) for id in ['7120', '7121', '9332']]

# Filtra per creare i dataset di training e test
train_data = filtered_data[~filtered_data['Unified_Id'].isin(excluded_books)]
test_data = filtered_data[filtered_data['Unified_Id'].isin(excluded_books)]

In [56]:
# shape dei dataset
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

Train data shape: (93879, 14)
Test data shape: (74, 14)


In [57]:
# numero di recensioni per libro nel dataset di test
print(test_data['Unified_Id'].value_counts())

# numero di utenti diversi che hanno recensito i libri nel dataset di test
print(test_data['User_id'].nunique())

Unified_Id
7120    25
7121    25
9332    24
Name: count, dtype: int64
48


In [58]:
# stampa tutti i record del dataset di test
print(test_data)

               User_id  Unified_Id  review/score                       Title  \
150      A106016KSI0YQ        7120           4.0           a town like alice   
151      A106016KSI0YQ        7121           4.0  a town like alice (import)   
153      A106016KSI0YQ        9332           5.0                    affinity   
3894    A137B77BFBHZBU        7120           5.0           a town like alice   
3895    A137B77BFBHZBU        7121           5.0  a town like alice (import)   
...                ...         ...           ...                         ...   
187478   AS6SH2JM2YY3H        7121           5.0  a town like alice (import)   
188119   ASPABROV8R7M5        9332           4.0                    affinity   
190505   AU6DIIDZK2OQM        7120           4.0           a town like alice   
190506   AU6DIIDZK2OQM        7121           4.0  a town like alice (import)   
194855   AX54G5AL870Q8        9332           5.0                    affinity   

                    profileName review/

In [59]:
# trasforma la data in anno
filtered_data['publishedYear'] = filtered_data['publishedDate'].astype(str).str[:4]
filtered_data['publishedYear'] = pd.to_numeric(filtered_data['publishedYear'], errors='coerce').astype('Int64')
filtered_data.drop(columns=['publishedDate'], inplace=True)

In [70]:
print("Colonne presenti in filtered_data:", filtered_data.columns)
print("Valori NaN in 'Unified_Id':", filtered_data['Unified_Id'].isna().sum())


Colonne presenti in filtered_data: Index(['User_id', 'Unified_Id', 'review/score', 'Title', 'profileName',
       'review/helpfulness', 'review/summary', 'review/text', 'description',
       'authors', 'publisher', 'categories', 'ratingsCount', 'publishedYear'],
      dtype='object')
Valori NaN in 'Unified_Id': 0


In [75]:
books_table = filtered_data[['Unified_Id', 'Title', 'authors', 'description', 'publisher', 'categories', 'publishedYear']].drop_duplicates(subset='Unified_Id')

books_table['Unified_Id_col'] = books_table['Unified_Id']  # Aggiungi una copia come colonna
books_table = books_table.set_index('Unified_Id_col')

# Conta il numero di libri nel dataset e verifica l'unicità di Unified_Id
n_books = books_table.shape[0]
print("Numero di libri:", n_books)

Numero di libri: 1774


In [76]:
print(books_table.columns)

Index(['Unified_Id', 'Title', 'authors', 'description', 'publisher',
       'categories', 'publishedYear'],
      dtype='object')


In [77]:
np.random.seed(42)  # Imposta il seed per risultati riproducibili, se desiderato
books_table['Price'] = np.random.randint(4, 26, size=books_table.shape[0])

print(books_table.head())

                Unified_Id                                  Title  \
Unified_Id_col                                                      
7120                  7120                      a town like alice   
7121                  7121             a town like alice (import)   
9332                  9332                               affinity   
14989                14989                 appointment in samarra   
14990                14990  appointment in samarra (signet books)   

                         authors  \
Unified_Id_col                     
7120             ['Nevil Shute']   
7121                        None   
9332            ['Sarah Waters']   
14989            ["John O'Hara"]   
14990            ["John O'Hara"]   

                                                      description  \
Unified_Id_col                                                      
7120            "A tale of love and war, follows its enterpris...   
7121                                                    

In [78]:
# NaN
print(books_table.isnull().sum())

Unified_Id         0
Title              0
authors          200
description      366
publisher        425
categories       324
publishedYear    172
Price              0
dtype: int64


In [79]:
assert books_table['Unified_Id'].is_unique, "Esistono ID duplicati nella tabella dei libri!"

In [81]:
# Salva la tabella dei libri in un file CSV
books_table.to_csv('data_preprocessed/books_table.csv')

In [82]:
# estrazione dei campi per la tabella delle recensioni
reviews_table = filtered_data[['User_id', 'Unified_Id', 'review/score', 'review/summary', 'review/text']]
reviews_table['review_id'] = range(1, len(reviews_table) + 1)
reviews_table['review_id_col'] = reviews_table['review_id']  

reviews_table = reviews_table.set_index('review_id_col')

print(reviews_table.head())

                     User_id  Unified_Id  review/score  \
review_id_col                                            
1              A106016KSI0YQ        7120           4.0   
2              A106016KSI0YQ        7121           4.0   
3              A106016KSI0YQ        9332           5.0   
4              A106016KSI0YQ       14989           4.0   
5              A106016KSI0YQ       14990           4.0   

                                       review/summary  \
review_id_col                                           
1                                Cold Comfort Cowtown   
2                                Cold Comfort Cowtown   
3              A pageturner that wins your confidence   
4                           Fear in a handful of dust   
5                           Fear in a handful of dust   

                                                     review/text  review_id  
review_id_col                                                                
1              It says something abou

C:\Users\Tommaso\AppData\Local\Temp\ipykernel_41660\3763055024.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_table['review_id'] = range(1, len(reviews_table) + 1)
C:\Users\Tommaso\AppData\Local\Temp\ipykernel_41660\3763055024.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_table['review_id_col'] = reviews_table['review_id']


In [85]:
print(reviews_table.shape)

(93953, 6)


In [83]:
# verifico i duplicati
duplicate_check = reviews_table.groupby(['User_id', 'Unified_Id']).size().reset_index(name='count')
duplicates = duplicate_check[duplicate_check['count'] > 1]

# Visualizza i duplicati
print("Combinazioni duplicate di User_id e Unified_Id:")
print(duplicates)

Combinazioni duplicate di User_id e Unified_Id:
Empty DataFrame
Columns: [User_id, Unified_Id, count]
Index: []


In [84]:
#Nan 
print(reviews_table.isnull().sum())

User_id           0
Unified_Id        0
review/score      0
review/summary    0
review/text       0
review_id         0
dtype: int64


In [87]:
# trasformo review/score in intero
reviews_table['review/score'] = reviews_table['review/score'].astype(int)

In [88]:
# Salva la tabella delle recensioni in un file CSV
reviews_table.to_csv('data_preprocessed/reviews_table.csv')